# Importing Libraries

In [3]:
import numpy as np
import pandas as pd
import cv2
import os
from patchify import patchify
from PIL import Image
from matplotlib import pyplot as plt
import tifffile
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras import layers

# Stacking and saving Images

In [1]:
# DIRECTORY = (r"C:\Users\tause\Advance_ANN\data")

# CATEGORIES = []
# for classes in os.listdir(DIRECTORY):
#     CATEGORIES += [classes]

# data = np.zeros((512,512,31))
# i=0
# for category in CATEGORIES:
#     path = os.path.join(DIRECTORY, category)
#     path = path + '\\' + CATEGORIES[i]
# #     print(path)
#     j=0
#     for img in os.listdir(path):
#         if(img==("Thumbs.db") or img==((CATEGORIES[i][:-2])+"RGB.bmp")):
#             continue
#         img_path = os.path.join(path, img)
#         arr = cv2.imread(img_path,0)
#         data[:,:,j] = arr
#         j+=1
# #     images = [Image.fromarray(data[:, :, i].astype('uint8')) for i in range(data.shape[2])]
# #     images[0].save((r"C:\Users\tause\Advance_ANN\Stacked\\")+(CATEGORIES[i][:-2]+"stacked.tiff"), format='TIFF', save_all=True, append_images=images[1:])
#     i+=1


In [ ]:
data.shape

In [ ]:
# plt.imshow(data[:,:,3],cmap='gray')

# Reading Stacked Images

In [4]:
DIRECTORY = (r"C:\Users\tause\Advance_ANN\stacked")

CATEGORIES = []
for classes in os.listdir(DIRECTORY):
    CATEGORIES += [classes]

# print(CATEGORIES)

data = []
i=0
for category in CATEGORIES:
    img_path = os.path.join(DIRECTORY, category)
    arr = cv2.imread(img_path,0)
    data.append(arr)

stacked = np.array(data)

In [5]:
stacked = np.transpose(stacked, (1, 2, 0))
stacked.shape

(512, 512, 32)

# Making Patches

In [6]:
def extract_blocks(image, block_size, stride):
    blocks = []
    image_height, image_width = image.shape[:2]
    num_blocks_x = image_width // stride
    num_blocks_y = image_height // stride
    for y in range(0,image_height-block_size[1]+1,stride):
        for x in range(0,image_width-block_size[0]+1,stride):
            block=image[y:y+block_size[1],x:x+block_size[0]]
            blocks.append(block)
    return blocks

In [7]:
patches = extract_blocks(stacked,(64,64),32)
len(patches)

225

# Labels

In [8]:
y = np.array(patches)
y.shape

(225, 64, 64, 32)

In [9]:
y = y.astype(float)

# Making Dataset

In [10]:
x1 = tf.keras.layers.AveragePooling2D(pool_size=(8, 8),strides=(8, 8), padding='valid',dtype='float64')
x1 = x1(y)
x1.shape

TensorShape([225, 8, 8, 32])

In [11]:
x2_1 = y[:,:,:,0:11].mean(axis = 3)
x2_2 = y[:,:,:,11:22].mean(axis = 3)
x2_3 = y[:,:,:,22:].mean(axis = 3)

In [12]:
x2 = np.concatenate([np.expand_dims(x2_1,3),np.expand_dims(x2_2,3),np.expand_dims(x2_3,3)],axis = 3)
x2.shape

(225, 64, 64, 3)

# PreProcessing for Model

In [13]:
print("Labels-shape",y.shape)
print("Input_Feature-1 shape",x1.shape)
print("Input_Feature-2 shape",x2.shape)

Labels-shape (225, 64, 64, 32)
Input_Feature-1 shape (225, 8, 8, 32)
Input_Feature-2 shape (225, 64, 64, 3)


# Reshaping

In [14]:
# labels = layers.Flatten(dtype='float64')(y)
# print(labels.shape)
# feat1 = layers.Flatten(dtype='float64')(x1)
# print(feat1.shape)
# feat2 = layers.Flatten(dtype='float64')(x2)
# print(feat2.shape)

In [16]:
X_interploated = tf.image.resize(x1, (64, 64), method='bilinear')
X_interploated.shape

TensorShape([225, 64, 64, 32])

In [17]:
X = tf.concat([X_interploated, x2], axis=-1)
X.shape

TensorShape([225, 64, 64, 35])

In [24]:
X = X/255
y = y/255

# Standardization

In [15]:
labels = labels/255
feat1 = feat1/255
feat2 = feat2/255

In [17]:
feat_all = np.zeros((225,(np.array(feat1.shape[1])+np.array(feat2.shape[1]))))
feat_all.shape

(225, 14336)

In [18]:
feat_all[:,0:np.array(feat1.shape[1])] = feat1
feat_all[:,np.array(feat1.shape[1]):] = feat2

In [19]:
print(feat_all.shape)
print(labels.shape)

(225, 14336)
(225, 131072)


# Model

In [ ]:
model = tf.keras.Sequential(
    [
        layers.Input(shape=(14336,)),
        layers.Dense(1024, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(131072,activation='sigmoid'),
    ]
)

In [ ]:
model.compile(optimizer = "Adam", loss ="binary_crossentropy",metrics=['accuracy'])
model.optimizer.lr = 0.001

In [ ]:
# history = model.fit(feat_all,labels,epochs = 50)

In [ ]:
# for i in range(225):
#     plt.subplot(15,15,i+1)
#     fig = plt.imshow(patches[i,:,:,1],cmap='gray')
#     fig.axes.get_xaxis().set_visible(False)
#     fig.axes.get_yaxis().set_visible(False)

In [20]:
encoding_dim = 24  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(14336,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(14336, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [21]:
encoder = keras.Model(input_img, encoded)

In [22]:
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [23]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [26]:
# autoencoder.fit(feat_all, feat_all, epochs=100, batch_size=256,)

In [47]:
t1 = np.zeros((58, 58, 128))
t1.shape

(58, 58, 128)

In [61]:
for i in range(2,128):
    for j in range(2,200):
        if i*i*j==430592:
            print("i",i,"j",j)

i 58 j 128
i 116 j 32


In [ ]:
# t2 = np.reshape(t1,(,,-1))

In [72]:
116-(53-1)

64

In [73]:
model = tf.keras.Sequential(
    [
        layers.Input((64, 64, 35)),
        layers.Conv2D(filters = 32,kernel_size = (3,3),strides=1,padding="valid",activation="relu"),
        #shape(batch_size, 62, 62, 32).
        layers.Conv2D(filters = 64,kernel_size = (3,3),strides=1,padding="valid",activation="relu"),
        #(batch_size, 60, 60, 32).
        layers.Conv2D(filters = 128,kernel_size = (3,3),strides=1,padding="valid",activation="relu"),
        #(batch_size, 58, 58, 128).
        
        layers.Reshape((116 ,116 ,32)),
        #(batch_size, 116, 116, 32).
        
        layers.Conv2D(filters = 32,kernel_size = (53,53),strides=1,padding="valid",activation="relu"),
        #(batch_size, 64, 64, 64).
   
        
#------------------------------------------------------------------------------------------------------        
#         layers.Conv2D(filters = 64,kernel_size = (5,5),strides=1,padding="valid",activation="relu"),
#         #(batch_size, 66, 66, 64).
        
#         layers.Conv2D(filters = 32,kernel_size = (3,3),strides=1,padding="valid",activation="relu"),
#         #(batch_size, 64, 64, 32).
    ]
)

In [74]:
model.compile(optimizer = "Adam", loss ="binary_crossentropy",metrics=['accuracy'])

In [75]:
print(X.shape)
y.shape

(225, 64, 64, 35)


(225, 64, 64, 32)

In [77]:
history = model.fit(X,y,epochs = 50)

Epoch 1/50
8/8 [==============================] - 311s 39s/step - loss: 0.4878 - accuracy: 0.0843
Epoch 2/50
8/8 [==============================] - 327s 41s/step - loss: 0.3278 - accuracy: 0.1313
Epoch 3/50
3/8 [==========>...................] - ETA: 2:55 - loss: 0.3275 - accuracy: 0.0686

KeyboardInterrupt: 